# Test Runs for Merging Changes

This notebook is used for testing changes made to the master branch, by running the models in base/model (which should be identical to the master branch) and changes/model (which should be the model run with your changes) 

Please follow the procedure described in the [readme](../../../README.md), and be critical if the tests performed here is enough to verify and validate the changes made. Furthermore, note the following:
- Make sure that the choice of addons is comparable, unless the change is a new addon
- If the changes are in files that by default are loaded in the base scenario (such as addons) you will need to apply the changes through options. Or, more simply, run the four tests manually in other folders and copy+paste the relevant MainResults to the paths expected by this script and append _lowtemphighspace and _hightemplowspace to the respective files. Skip to the analysis section below in that case.
- If a GamsExecutionError occur in the cell running Balmorel, check the _gams_py_gjo0.lst file, or maybe try to run the scenario in GAMS studio for easier debugging
- All tests may take around 20 minutes.

The [pybalmorel](https://pypi.org/project/pybalmorel/) package is required to run this notebook. In a terminal, create a new python environment, activate it and install version 0.2.2 by running the following command:
```
pip install pybalmorel==0.2.2
```
Documentation on creating and activating virtual environments for standalone python can be found [here](https://docs.python.org/3/library/venv.html) and for a conda installation [here](https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html)

In [101]:
from pybalmorel import MainResults, IncFile
from gams import GamsWorkspace
import pandas as pd
import numpy as np
import os

## Scenario and test case names
scenarios = ['base', 'changes']
test_cases = ['lowtemphighspace', 'hightemplowspace']

## Run Balmorel

The following section runs the base and changes scenario at different resolutions:
1. Low temporal resolution (Two seasons with four hours each) and high spatial resolution (the existing resolution in each C file)
2. High temporal resolution (Four seasons with all hours) and low spatial resolution (only Denmark and Norway) 

In [ ]:
### 1.0 Prepare .inc-file formats for test cases
C = IncFile(name='C',
            prefix="SET C(CCC)  'Countries in the simulation'\n/\n",
            suffix="\n/;",
            )
Y = IncFile(name='Y',
            prefix="SET  Y(YYY) 'Years in the simulation'\n/\n",
            suffix="\n/;",
            )
S = IncFile(name='S',
            prefix="SET S(SSS)  'Seasons in the simulation'\n/\n",
            suffix="\n/;",
            )
T = IncFile(name='T',
            prefix="SET T(TTT)  'Time periods within a season in the simulation'\n/\n",
            suffix="\n/;",
            )

### 1.1 Running Balmorel Test Cases
for scenario in scenarios:
    for test_case in test_cases:
        
        # Set resolutions
        C.path ='../../../%s/data'%scenario
        Y.path ='../../../%s/data'%scenario
        S.path ='../../../%s/data'%scenario
        T.path ='../../../%s/data'%scenario
        if test_case == 'lowtemphighspace':
            # keep the country resolution of the base and change in this case
            Y.body = "2030, 2050"
            S.body = "S01, S26"
            T.body = "T073, T079, T085, T091"
            
            # rename original temporal .inc-files so they're not lost
            try:
                os.rename('../../../%s/data/Y.inc'%scenario,
                        '../../../%s/data/Y_old.inc'%scenario)
                os.rename('../../../%s/data/S.inc'%scenario,
                        '../../../%s/data/S_old.inc'%scenario)
                os.rename('../../../%s/data/T.inc'%scenario,
                        '../../../%s/data/T_old.inc'%scenario)
            except (FileNotFoundError, FileExistsError):
                print('Possibly already renamed old resolution .inc-files')
                
        elif test_case == 'hightemplowspace':
            C.body = "DENMARK, NORWAY"
            Y.body = "2030, 2050"
            S.body = "S01, S14, S27, S40"
            T.body = "T001*T168"
            
            # rename original C .inc-file so it's not lost
            try:
                os.rename('../../../%s/data/C.inc'%scenario,
                        '../../../%s/data/C_old.inc'%scenario)
            except (FileNotFoundError, FileExistsError):
                print('Possibly already renamed old resolution .inc-files')
                
            C.save()
        Y.save()
        S.save()
        T.save()
        
        # Run Balmorel
        ws = GamsWorkspace(working_directory='../../../%s/model'%scenario)

        job = ws.add_job_from_file(os.path.join(os.path.abspath('../../../%s/model'%scenario), 'Balmorel'))
        out = job.run()
        
        # Check feasibility
        with open('../../../%s/model/_gams_py_gjo0.lst'%scenario, 'r') as f:
            output = pd.Series(f.readlines())

        output = output[output.str.find('LP status') != -1] # Find all status
        all_feasible = output[output.str.find('infeasible') != -1].empty # Check if none are infeasible
        if not(all_feasible):
            raise Exception('Model run infeasible!')
        
        # Rename MainResults
        try:
            os.rename('../../../%s/model/MainResults.gdx'%scenario,
                    '../../../%s/model/MainResults_%s.gdx'%(scenario, test_case))
        except FileExistsError:
            print('Previous run existed, overwriting')
            os.replace('../../../%s/model/MainResults.gdx'%scenario,
                    '../../../%s/model/MainResults_%s.gdx'%(scenario, test_case))
        

## Analysis

General results are compared between scenarios for each test case and KPI's are calculated on production of commodities and capacities. 

In [30]:
scenarios = ['base', 'base_changes']

### 2.0 Load Results
mr = MainResults(files=['MainResults_%s.gdx'%test_case for test_case in test_cases]*2,
                 paths=['../../../%s/model'%scenario for scenario in scenarios for i in range(2) ],
                 scenario_names=['%s_%s'%(scenario, test_case) for scenario in scenarios for test_case in test_cases])

--- Warning: The GAMS version (45.5.0) differs from the API version (45.7.0).


In [93]:
### 2.1 Calculate Difference in capduction 
pro = mr.get_result('PRO_YCRAGF').pivot_table(index='Year',
                                              columns=['Scenario', 'Commodity'],
                                              values='Value',
                                              aggfunc='sum')

print('All production [TWh]:\n---------------------\n', pro.to_string(), '\n')
for test_case in test_cases:
    print('\nDifference in production in TWh between %s and %s in for %s'%tuple(scenarios + [test_case]),
          '\n-------------------------------------------------------------\n')
    print((pro['%s_%s'%(scenarios[1], test_case)] - pro['%s_%s'%(scenarios[0], test_case)]).to_string() + '\n')

All production [TWh]:
---------------------
 Scenario  base_changes_hightemplowspace                        base_changes_lowtemphighspace                        base_hightemplowspace                       base_lowtemphighspace                      
Commodity                   ELECTRICITY        HEAT   HYDROGEN                   ELECTRICITY        HEAT   HYDROGEN           ELECTRICITY       HEAT   HYDROGEN           ELECTRICITY       HEAT   HYDROGEN
Year                                                                                                                                                                                                       
2030                                NaN         NaN        NaN                    204.885725  141.036449  31.666812                   NaN        NaN        NaN            146.877826  97.535908   7.353390
2050                         251.124885  140.016999  53.919089                    246.528420  143.795472  52.446507            180.074473  

In [112]:
### 2.2 Calculate Difference in Generation Capacities
cap = mr.get_result('G_CAP_YCRAF')
cap = cap[(cap.Technology != 'INTRASEASONAL-ELECT-STORAGE') &\
        (cap.Technology != 'INTRASEASONAL-HEAT-STORAGE') &\
        (cap.Technology != 'INTERSEASONAL-HEAT-STORAGE') &\
        (cap.Technology != 'H2-STORAGE')].pivot_table(index='Year',
                                                            columns=['Scenario', 'Commodity'],
                                                            values='Value',
                                                            aggfunc='sum').fillna(0)

print('All generation capacities [GW]:\n-------------------------------\n', cap.to_string(), '\n')
for test_case in test_cases:
    print('\nDifference in generation capacities in GW between %s and %s in for %s:'%tuple(scenarios + [test_case]),
          '\n-------------------------------------------------------------')
    print((cap['%s_%s'%(scenarios[1], test_case)] - cap['%s_%s'%(scenarios[0], test_case)]).to_string() + '\n')

All generation capacities [GW]:
-------------------------------
 Scenario  base_changes_hightemplowspace                      base_changes_lowtemphighspace                      base_hightemplowspace                                  base_lowtemphighspace                                 
Commodity                   ELECTRICITY       HEAT  HYDROGEN                   ELECTRICITY       HEAT  HYDROGEN            BIOMETHANE ELECTRICITY       HEAT  HYDROGEN            BIOMETHANE ELECTRICITY       HEAT  HYDROGEN
Year                                                                                                                                                                                                                         
2030                           0.000000   0.000000  0.000000                      62.22878  27.296902  4.084225                   0.0    0.000000   0.000000  0.000000                   0.0   35.418515  17.154851  0.877514
2050                          92.052875  30.571

In [109]:
### 2.3 Calculate Difference in Storage Capacities
cap = mr.get_result('G_STO_YCRAF').pivot_table(index='Year',
                                        columns=['Scenario', 'Commodity'],
                                        values='Value',
                                        aggfunc='sum').fillna(0)

print('All storage capacities [GWh]:\n-----------------------------\n', cap.to_string(), '\n')
for test_case in test_cases:
    print('\nDifference in storage capacities in GWh between %s and %s in for %s:'%tuple(scenarios + [test_case]),
          '\n-------------------------------------------------------------')
    print((cap['%s_%s'%(scenarios[1], test_case)] - cap['%s_%s'%(scenarios[0], test_case)]).to_string() + '\n')

All storage capacities [GWh]:
-----------------------------
 Scenario  base_changes_hightemplowspace                         base_changes_lowtemphighspace                         base_hightemplowspace             base_lowtemphighspace            
Commodity                   ELECTRICITY        HEAT    HYDROGEN                   ELECTRICITY        HEAT    HYDROGEN           ELECTRICITY        HEAT           ELECTRICITY        HEAT
Year                                                                                                                                                                                     
2030                                0.0    0.000000    0.000000                       82224.0  395.604526  238.621993                   0.0    0.000000               82224.0  671.524689
2050                            82224.0  500.063685  342.978697                       82224.0  550.614625  238.621993               82224.0  263.948093               82224.0  692.441894 


Differ

In [110]:
### 2.4 Calculate Difference in Transmission Capacities
cap = mr.get_result('X_CAP_YCR').pivot_table(index='Year',
                                        columns=['Scenario'],
                                        values='Value',
                                        aggfunc=lambda x: np.sum(x)/2).fillna(0) # Remember to divide by two

print('All electricity transmission capacities [GW]:\n---------------------------------------------\n', cap.to_string(), '\n')
for test_case in test_cases:
    print('\nDifference in electricity transmission capacities in GW between %s and %s in for %s:'%tuple(scenarios + [test_case]),
          '\n-------------------------------------------------------------')
    print((cap['%s_%s'%(scenarios[1], test_case)] - cap['%s_%s'%(scenarios[0], test_case)]).to_string() + '\n')

All electricity transmission capacities [GW]:
---------------------------------------------
 Scenario  base_changes_hightemplowspace  base_changes_lowtemphighspace  base_hightemplowspace  base_lowtemphighspace
Year                                                                                                                
2030                            0.00000                      11.077000               0.000000              15.069722
2050                           18.98553                      14.256002              15.986844              15.069722 


Difference in electricity transmission capacities in GW between base and base_changes in for lowtemphighspace: 
-------------------------------------------------------------
Year
2030   -3.992722
2050   -0.813720


Difference in electricity transmission capacities in GW between base and base_changes in for hightemplowspace: 
-------------------------------------------------------------
Year
2030    0.000000
2050    2.998686

